# Project 5: Minesweeper Logic Agent
## Jared Rand

### Purpose: To create a bot that can play (and hopefully solve) Minesweeper. 


# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# QUICK NOTE: ONLY WORKS WITH MINESWEEPER.ONLINE AS OF RIGHT NOW. I WANT TO ADD OTHER SITES LATER.
# ALSO: HERE'S A VIDEO OF MY BOT SOLVING THE 9x9: https://www.youtube.com/watch?v=eHvEj_88IqE&feature=youtu.be
### I only just realized there's the same clip in there twice, but there's still 4 others : )
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# Introduction: 
 
 In this project we were meant to create a bot that is capable of playing minesweeper. 
 Just FYI, this gif is not my bot
 
 <img src="img/bettermine.gif">
 
 Minesweeper is a game that starts with a n*n square board of squares. Each square may contain nothing, a mine, or a number that indicates the number of mines next to that square. You need to reveal all the squares that aren't mines, and flag all the ones that are. It is a pretty straightforward game, but it is difficult if you don't know what's going on. I still have yet to win a game of minesweeper by myself. 

# Discussion Of Solution

This project took me a pretty good deal of time, so I'm glad that I got started on it early. For a project that took so long, I'm glad it was at least an entertaining one. 

I started out by making sure that the bot could find the board, and that it could register when the board changed after each click. 

In [4]:
def get_blanks(smile, blank):
    isonscreen = None
    i = 0
    while isonscreen == None:   # makes sure the board is on the main screen
        isonscreen = pyautogui.locateOnScreen(smile, grayscale=False, confidence=.5)
        i += 1
        if i > 1:
            print("\nBoard Not Found -- Please Bring Into View")
        if i > 10:
            print("\nCould Not Find Board -- Aborting...")
            sleep(3)
            quit()
        sleep(3)
        
    Blanks = pyautogui.locateAllOnScreen(blank, grayscale=True)
    squares = list(Blanks)
    return squares


This is the method that finds the board. It's a slightly modified version of the code given to us. I made 2 changes, first being that it searches the screen 10 times if it can't find the board (giving me time to bring the board on screen if I forgot), and the second is removing the search range, so it can find a minesweeper board of any size (so long as it fits on your screen).


In [5]:
class Square:

    def __init__(self, left, top, width, height, name='square'):
        self.name = name
        self.left = left
        self.top = top
        self.width = width
        self.height = height
        self.type = 'blank'  # blank, empty, bomb?, one, two, three, four, five?
        self.likelihood = 0
        self.flagged = False


        self.surroundings = [[None, None, None, None, None, None, None, None]]
        self.added = []
                        # Topleft, Top, Topright, Left, Right, Bottomleft, Bottom, Bottomright
                        #    0      1       2       3     4        5          6         7


This is the class I made for each of the squares. It keeps track of all the important bits that I needed. Especially the type of square it was, the likelihood it was a bomb, and its surroundings. 


The way that I updated the board was, frankly, a bit of a mess. If I find the time I'll try to clean it up a little bit and hopefully make it run faster, and easier to read. Essentially what I did was create a dict of all the squares, the key being the left coordinate combined with the top coordinate (100 + 50 = 10050), and it returns the class at that position. That way when I searched for each type of square, I could easily figure out what square it was referring to. 



In [6]:
    for i in range(len(ones)):
        s = str(ones[i].left-1) + str(ones[i].top-1)
        holder[s].type = 'one'
        holder[s].likelihood = -1

        count = 0
        keepaneyeon = []
        for j in range(len(holder[s].surroundings[0])):
            if holder[s].surroundings[0][j] != None and holder[s].surroundings[0][j].type == 'blank':
                count += 1
                keepaneyeon.append(holder[s].surroundings[0][j])
            if holder[s].surroundings[0][j] != None and holder[s].surroundings[0][j].type == 'flagged':
                count += 1
                keepaneyeon.append(holder[s].surroundings[0][j])
        if len(keepaneyeon) == 1:
            for j in range(len(keepaneyeon)):
                if keepaneyeon[j].flagged != True:
                    keepaneyeon[j].flagged = True
                    pyautogui.rightClick(x=keepaneyeon[j].left + 10, y=keepaneyeon[j].top + 10)
                    for k in range(len(keepaneyeon[j].surroundings[0])):
                        if keepaneyeon[j].surroundings[0][k] is None:
                            continue
                        if keepaneyeon[j].surroundings[0][k].type == 'blank' or keepaneyeon[j].surroundings[0][k].type == 'empty':
                            continue
                        elif keepaneyeon[j].surroundings[0][k].type == 'one' or \
                                keepaneyeon[j].surroundings[0][k].type == 'two' or \
                                keepaneyeon[j].surroundings[0][k].type == 'three' or \
                                keepaneyeon[j].surroundings[0][k].type == 'four':
                            pyautogui.click(x=keepaneyeon[j].surroundings[0][k].left + 10, y=keepaneyeon[j].surroundings[0][k].top + 10)

NameError: name 'ones' is not defined

This is part does a couple of things. It keeps track of the number of blank or flagged squares next to it (A square is 2 with only two blank squares, then both of those blanks are mines.), it will flag blank squares, and it will click all the number squares afterwards(because this reveals more blank squares)

This part has 2 main drawbacks, 
    1. It looks ugly and it's hard to read. There's so much going on in there that I get lost half the time while reading it.
    2. It's what bogs the rest of the program down. I know that there is an easier and quicker way to do this, which I'll talk about later, but this is the way I did it for now. The biggest problem is that I need to do this for each number tile, and each loop has 3 other loops inside it (Most of them go for less than 8 iterations, but still, it adds up), so that really adds on the runtime.    
    
One last thing, it decides what to click in 2 different ways. The first is that it flags definite mines and clicks around it, revealing more squares. The second way is that it calculates what it belives to be the least risky square to click, which I could do a better job at calculating, because it still hits mines sometimes. I also read online that the best first move to make is to click the four corners of the board, but I'm beginning to believe that's not true, because it still manages to hit a large number of mines in the corners. 

# Testing Design and Expected Results

I'm actually somewhat pleased with how this bot turned out. It's not the prettiest thing, but it can solve a 9x9 pretty okay! Luckily the minesweeper website I used has a timer built in, so I didn't need to time it myself. 

I based how it worked in 4 categories:
    1. Found The Solution
    2. Came Close (clicked on the final mine, <1 remaining)
    3. Clicked On A Bomb In The Corner (First 4 Moves)
    4. Just No Where Close (Lost)
    


# Actual Results:

Out of 15 runs for each, these are the results I got:

----When clicking all 1 corners as first move 

    1. 5 Times
    2. 0 Times
    3. 0 Times
    4. 10 Times
    
----When clicking all 2 corners as first move 

    1. 6 Times
    2. 0 Times
    3. 1 Time
    4. 8 Times
    
----When clicking all 3 corners as first move 

    1. 3 Times
    2. 0 Times
    3. 6 Times
    4. 6 Times

----When clicking all 4 corners as first move 

    1. 5 Times
    2. 3 Times
    3. 6 Times
    4. 1 Time
    
----(I also read online about another starting moveset, where you click in a corner, the middle of one of the edges, and somewhere in the center of the board. These are the results from clicking the left corner, middle of the top edge, and right in the middle of the board.)
    
    1. 6 Times
    2. 0 Times
    3. 0 Times
    4. 9 Time
    

# Discussion Of Results:

I should probably run these more times just to be sure, but it seems like it finds the solution best when it only clicks the top 2 corners. It's not a significant difference, but it is a difference. I think this method would work best if it kept track of how many were revealed by a corner click. Like, if it only revealed a single number, then it would click another corner, but if it revealed a lot of the board, then maybe it just wouldn't click the corners. This or the 5th solution where you click in the middle would be the better solutions.

I'm happy it was able to solve the 9x9, but the amount of times it lost by clicking on a mine in the corner was too many, and that's a part that I need to fix. Along with how it determines which is the safest square to click on, because that's most of the reason why it was able to come close 3 times, but lose right at the end, because it misjudged which square was the mine. Overall though, not the worst bot I've ever made though! I'll try to make it more efficient when I have the time, and when I do I'll either update it here or put it on my github (Or Both)(https://github.com/NotEnoughMilk)

# Link To Azure Notebook:

https://notebooks.azure.com/randja/projects/minesweeper-logic-agent

# The Program Below Won't Run In Here, But I'll Throw It Down There Anyway:

## THIS ONE START'S IT \/

In [ ]:
from locateboard import *
from square import *
from update_board import *
from play import *


typeofmines = input("What Online Minesweeper Are You Using?\n1. Minesweeper.online\n2. Minesweeperonline.com\n\n")
# smile, blank, one, two, three, four, empty
if typeofmines == 2:
    tiles = ['img/minesweeperonline.com/smile2.png', 'img/minesweeperonline.com/blank.png', 'img/minesweeperonline.com/one.png',
             'img/minesweeperonline.com/two.png', 'img/minesweeperonline.com/three.png', 'img/minesweeperonline.com/four.png',
             'img/minesweeperonline.com/empty.png', 'img/minesweeperonline.com/flag.png', 'img/minesweeperonline.com/bomb.png']
else:
    tiles = ['img/minesweeper.online/smile1.png', 'img/minesweeper.online/blank1.png', 'img/minesweeper.online/one1.png',
             'img/minesweeper.online/two1.png', 'img/minesweeper.online/three1.png', 'img/minesweeper.online/four1.png',
             'img/minesweeper.online/empty1.png', 'img/minesweeper.online/flag.png', 'img/minesweeper.online/bomb.png']

board = get_blanks(tiles[0], tiles[1])
squares = []
for i in range(len(board)):
    me = Square(board[i][0], board[i][1], board[i][2], board[i][3], name='S{}'.format(i + 1))
    squares.append(me)
boardholder = {}

# ## This parts a bit of a mess, but it creates the surroundings################################################
boardss = [[]]
b = []
start = 0
left = squares[0].left
right = 0
for i in range(len(squares)):
    if squares[i].left == left and i != 0:
        boardss.insert(start, b)
        b = []
        start += 1
    b.append(squares[i])
    s = str(squares[i].left) + str(squares[i].top)  # creates a dict of all the squares with the key as their
    boardholder[s] = squares[i]                     # left + top coords (this makes for easier updating)
boardss.insert(start, b)
b = []

for i in range(len(boardss)-2):  # -2 because it creates an empty bottom layer that doesnt matter all that much
    for j in range(len(boardss[i])-1):
        if i > 0: # if not top layer
            if j > 0:
                boardss[i][j].surroundings[0][0] = boardss[i - 1][j - 1]  # topleft
            boardss[i][j].surroundings[0][1] = boardss[i - 1][j]  # top
            if j < len(boardss[i])-1:
                boardss[i][j].surroundings[0][2] = boardss[i - 1][j + 1]  # topright
        if j > 0:
            boardss[i][j].surroundings[0][3] = boardss[i][j - 1]  # left
        if j < len(boardss[i])-1:
            boardss[i][j].surroundings[0][4] = boardss[i][j + 1]  # right
        if i < len(boardss)-2:  # if not bottom layer
            if j > 0:
                boardss[i][j].surroundings[0][5] = boardss[i + 1][j - 1]  # bottomleft
            boardss[i][j].surroundings[0][6] = boardss[i + 1][j]  # bottom
            if j < len(boardss[i])-1:
                boardss[i][j].surroundings[0][7] = boardss[i + 1][j + 1]  # bottomright
#####################################################################

[gameLocX, gameLocY, _, _] = pyautogui.locateOnScreen(tiles[0], grayscale=False, confidence=.5)

# originally (tiles, squares, ... (in case it mattered)
update(tiles, boardss, boardholder, gameLocX, gameLocY)
pyautogui.click(x=squares[0].left + 10, y=squares[0].top + 10)  # Slaps the four corners, opens up a lot of the board
update(tiles, boardss, boardholder, gameLocX, gameLocY)
if boardss[0][len(boardss[0])-1].flagged is False:
    pyautogui.click(x=boardss[0][len(boardss[0])-1].left + 10, y=boardss[0][len(boardss[0])-1].top + 10)
update(tiles, boardss, boardholder, gameLocX, gameLocY)
if boardss[len(boardss)-2][0].flagged is False:
    pyautogui.click(x=boardss[len(boardss)-2][0].left + 10, y=boardss[len(boardss)-2][0].top + 10)
update(tiles, boardss, boardholder, gameLocX, gameLocY)
if boardss[len(boardss)-2][len(boardss[0])-1].flagged is False:
    pyautogui.click(x=boardss[len(boardss)-2][len(boardss[0])-1].left + 10, y=boardss[len(boardss)-2][len(boardss[0])-1].top + 10)
update(tiles, boardss, boardholder, gameLocX, gameLocY)
update(tiles, boardss, boardholder, gameLocX, gameLocY)
play(tiles, boardss, boardholder, gameLocX, gameLocY)


# Then It Moves To This One After: \/

In [ ]:
from locateboard import *
from update_board import *
import pyautogui


def play(tiles, board, boardholder, gameLocX, gameLocY):
    i = 0
    scanXStart = board[0][0].left - 5
    scanYStart = board[0][0].top - 10
    scanXEnd = board[len(board) - 2][len(board[0]) - 1].left + board[len(board) - 2][len(board[0]) - 1].width + 5
    scanYEnd = board[len(board) - 2][len(board[0]) - 1].top + board[len(board) - 2][len(board[0]) - 1].height + 10
    noend = True
    previous = ""
    while noend:
        blanks = list(pyautogui.locateAllOnScreen(tiles[1], grayscale=True, region=(scanXStart, scanYStart, scanXEnd, scanYEnd)))
        holder = update(tiles, board, boardholder, gameLocX, gameLocY)
        bomb = getleastlikely(blanks, holder)

        if previous is bomb:
            blanks.remove(bomb)
            bomb = getleastlikely(blanks, holder)
        previous = bomb
        update(tiles, board, boardholder, gameLocX, gameLocY)
        if bomb.flagged == True:
            continue
        pyautogui.click(x=bomb.left + 10, y=bomb.top + 10)

        i += 1
        if i == 20:
            noend = False
